In [1]:
# All Libraries required for this lab are listed below. The libraries pre-installed on Skills Network Labs are commented.
# !mamba install -qy pandas==1.3.4 numpy==1.21.4 seaborn==0.9.0 matplotlib==3.5.0 scikit-learn==0.20.1
# Note: If your environment doesn't support "!mamba install", use "!pip install"

In [4]:
# Surpress warnings:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [3]:
#you are running the lab in your  browser, so we will install the libraries using ``piplite``
import piplite
await piplite.install(['pandas'])
await piplite.install(['numpy'])


In [5]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.metrics import jaccard_score
from sklearn.metrics import f1_score
from sklearn.metrics import log_loss
from sklearn.metrics import confusion_matrix, accuracy_score
import sklearn.metrics as metrics

In [6]:
from pyodide.http import pyfetch

async def download(url, filename):
    response = await pyfetch(url)
    if response.status == 200:
        with open(filename, "wb") as f:
            f.write(await response.bytes())

In [7]:
path='https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML0101EN-SkillUp/labs/ML-FinalAssignment/Weather_Data.csv'

In [8]:
await download(path, "Weather_Data.csv")
filename ="Weather_Data.csv"

In [9]:
df = pd.read_csv("Weather_Data.csv")
df.head()

,Date,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2/1/2008,19.5,22.4,15.6,6.2,0.0,W,41,S,SSW,...,92,84,1017.6,1017.4,8,8,20.7,20.9,Yes,Yes
1,2/2/2008,19.5,25.6,6.0,3.4,2.7,W,41,W,E,...,83,73,1017.9,1016.4,7,7,22.4,24.8,Yes,Yes
2,2/3/2008,21.6,24.5,6.6,2.4,0.1,W,41,ESE,ESE,...,88,86,1016.7,1015.6,7,8,23.5,23.0,Yes,Yes
3,2/4/2008,20.2,22.8,18.8,2.2,0.0,W,41,NNE,E,...,83,90,1014.2,1011.8,8,8,21.4,20.9,Yes,Yes
4,2/5/2008,19.7,25.7,77.4,4.8,0.0,W,41,NNE,W,...,88,74,1008.3,1004.8,8,8,22.5,25.5,Yes,Yes


In [10]:
df_sydney_processed = pd.get_dummies(data=df, columns=['RainToday', 'WindGustDir', 'WindDir9am', 'WindDir3pm'])

In [11]:
df_sydney_processed.head()

,Date,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,...,WindDir3pm_NNW,WindDir3pm_NW,WindDir3pm_S,WindDir3pm_SE,WindDir3pm_SSE,WindDir3pm_SSW,WindDir3pm_SW,WindDir3pm_W,WindDir3pm_WNW,WindDir3pm_WSW
0,2/1/2008,19.5,22.4,15.6,6.2,0.0,41,17,20,92,...,0,0,0,0,0,1,0,0,0,0
1,2/2/2008,19.5,25.6,6.0,3.4,2.7,41,9,13,83,...,0,0,0,0,0,0,0,0,0,0
2,2/3/2008,21.6,24.5,6.6,2.4,0.1,41,17,2,88,...,0,0,0,0,0,0,0,0,0,0
3,2/4/2008,20.2,22.8,18.8,2.2,0.0,41,22,20,83,...,0,0,0,0,0,0,0,0,0,0
4,2/5/2008,19.7,25.7,77.4,4.8,0.0,41,11,6,88,...,0,0,0,0,0,0,0,1,0,0


In [12]:
df_sydney_processed.replace(['No', 'Yes'], [0,1], inplace=True)

In [13]:
df_sydney_processed.head()

,Date,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,...,WindDir3pm_NNW,WindDir3pm_NW,WindDir3pm_S,WindDir3pm_SE,WindDir3pm_SSE,WindDir3pm_SSW,WindDir3pm_SW,WindDir3pm_W,WindDir3pm_WNW,WindDir3pm_WSW
0,2/1/2008,19.5,22.4,15.6,6.2,0.0,41,17,20,92,...,0,0,0,0,0,1,0,0,0,0
1,2/2/2008,19.5,25.6,6.0,3.4,2.7,41,9,13,83,...,0,0,0,0,0,0,0,0,0,0
2,2/3/2008,21.6,24.5,6.6,2.4,0.1,41,17,2,88,...,0,0,0,0,0,0,0,0,0,0
3,2/4/2008,20.2,22.8,18.8,2.2,0.0,41,22,20,83,...,0,0,0,0,0,0,0,0,0,0
4,2/5/2008,19.7,25.7,77.4,4.8,0.0,41,11,6,88,...,0,0,0,0,0,0,0,1,0,0


In [14]:
df_sydney_processed.drop('Date',axis=1,inplace=True)

In [15]:
df_sydney_processed = df_sydney_processed.astype(float)

In [18]:
features = df_sydney_processed.drop(columns='RainTomorrow', axis=1)
Y = df_sydney_processed['RainTomorrow']

In [20]:
#Linear Regression
#Q1)
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split( features, Y, test_size=0.2, random_state=10)
print ('Train set:', x_train.shape,  y_train.shape)
print ('Test set:', x_test.shape,  y_test.shape)

Train set: (2616, 66) (2616,)
Test set: (655, 66) (655,)


In [27]:
#Q2)
from sklearn import linear_model
LinearReg = linear_model.LinearRegression()
LinearReg.fit(x_train, y_train)

LinearRegression()

In [28]:
#Q3)
predictions = LinearReg.predict(x_test)

In [36]:
#Q4)
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
LinearRegression_MAE = mean_absolute_error(y_test, predictions)
LinearRegression_MSE = mean_squared_error(y_test, predictions)
LinearRegression_R2 = r2_score(y_test, predictions)

In [30]:
#Q5)
Report = pd.DataFrame({
    'Metric': ['MAE', 'MSE', 'R2'],
    'Value': [LinearRegression_MAE, LinearRegression_MSE, LinearRegression_R2]
})
# Display the DataFrame
print(Report)

  Metric     Value
0    MAE  0.256319
1    MSE  0.115721
2     R2  0.427130


In [34]:
#KNN
#Q6)
#from sklearn import linear_model
#KNN = linear_model.KNeighborsClassifier()
KNN = KNeighborsClassifier(n_neighbors = 4).fit(x_train,y_train)
KNN.fit(x_train, y_train)

KNeighborsClassifier(n_neighbors=4)

In [35]:
#Q7)
predictions = KNN.predict(x_test)

In [37]:
#Q8)
KNN_Accuracy_Score = accuracy_score(y_test, predictions)
KNN_JaccardIndex = jaccard_score(y_test, predictions)
KNN_F1_Score = f1_score(y_test, predictions)

In [38]:
Report = pd.DataFrame({
    'Metric': ['AS', 'JC', 'F1'],
    'Value': [KNN_Accuracy_Score, KNN_JaccardIndex, KNN_F1_Score]
})
# Display the DataFrame
print(Report)

  Metric     Value
0     AS  0.818321
1     JC  0.425121
2     F1  0.596610


In [43]:
#Decision Tree
#Q9)
deciTree = DecisionTreeClassifier(criterion="entropy", max_depth = 4)
deciTree.fit(x_train,y_train)

DecisionTreeClassifier(criterion='entropy', max_depth=4)

In [44]:
#Q10)
predTree = deciTree.predict(x_test)

In [45]:
#Q11)
Tree_Accuracy_Score = accuracy_score(y_test, predTree)
Tree_JaccardIndex = jaccard_score(y_test, predTree)
Tree_F1_Score = f1_score(y_test, predTree)

In [46]:
Report = pd.DataFrame({
    'Metric': ['AS', 'JC', 'F1'],
    'Value': [Tree_Accuracy_Score, Tree_JaccardIndex, Tree_F1_Score]
})
# Display the DataFrame
print(Report)

  Metric     Value
0     AS  0.818321
1     JC  0.480349
2     F1  0.648968


In [47]:
#Logistic Regression
#Q12)
x_train, x_test, y_train, y_test = train_test_split( features, Y, test_size=0.2, random_state=1)

In [48]:
#Q13)
LR = LogisticRegression(C=0.01, solver='liblinear').fit(x_train,y_train)

In [49]:
#Q14)
predictions = LR.predict(x_test)
predict_proba = LR.predict_proba(x_test)

In [50]:
#Q15)
LR_Accuracy_Score = accuracy_score(y_test, predictions)
LR_JaccardIndex = jaccard_score(y_test, predictions)
LR_F1_Score = f1_score(y_test, predictions)
LR_Log_Loss = log_loss(y_test, predictions)
Report = pd.DataFrame({
    'Metric': ['AS', 'JC', 'F1', 'LL'],
    'Value': [LR_Accuracy_Score, LR_JaccardIndex, LR_F1_Score, LR_Log_Loss]
})
# Display the DataFrame
print(Report)

  Metric     Value
0     AS  0.827481
1     JC  0.484018
2     F1  0.652308
3     LL  6.218218


In [55]:
#SVM
#Q16)
SVM = svm.SVC(kernel='rbf')
SVM.fit(x_train,y_train)

SVC()

In [57]:
#Q17)
predictions = SVM.predict(x_test)

In [58]:
#Q18)
SVM_Accuracy_Score = accuracy_score(y_test, predictions)
SVM_JaccardIndex = jaccard_score(y_test, predictions)
SVM_F1_Score = f1_score(y_test, predictions)

In [54]:
#Q19)
Report = pd.DataFrame({
    'Metric': ['AS', 'JC', 'F1'],
    'Value': [LR_Accuracy_Score, LR_JaccardIndex, LR_F1_Score]
})
# Display the DataFrame
print(Report)